<a href="https://colab.research.google.com/github/hasune613/hello-world/blob/main/Re_pandas_100knocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データサイエンス100本ノック（構造化データ加工編） - Python
for Google Colab

### 【注意】オリジナル版との変更点
1. Google Colab ではDockerを使えないので、2020.06.18時点の 100knocks-preprocess/docker/work/data にあるCSVファイルをPostgreSQLから入手する代わりに使います。
2. オリジナルのCSVデータ, geocode.csvの'latitude'列名の初めにスペースが入っていたため、それを削除しました。

   オリジナル(100knocks-preprocess ver.1.0): ' latitude' --> 'latitude'
   

4. オリジナルの解答を元に、必要のないlibraryはimportせず、そして必要なlibraryをAzureNotebookでインストールするように最初のセルを改変してあります。
5. また、SQLではなく上記CSVからデータを読み込むように'はじめに'の最初のセルを改変してあります。

### はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートと~~データベース（PostgreSQL）~~ 100knocks-preprocess/docker/work/data にあるCSVファイルからのデータ読み込みを行います。geocode.csvに変更を加えたため、またgit cloneをするとColabの簡便さを損なうため、noguhiro2002のgithubレポジトリから直接読み込みます。
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能)
- オリジナルの解答例を元に、必要なライブラリーをpipでインストールします。
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# pipでオリジナルの解答に必要なライブラリーをインポート
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler


# データを github/noguhiro2002/100knocks-preprocess/work/data フォルダよりDataframe形式でRead
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_category = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/category.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')
df_store = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/store.csv')
df_geocode = pd.read_csv('https://raw.githubusercontent.com/noguhiro2002/100knocks-preprocess_ForColab-AzureNotebook/master/data/geocode.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# pipでオリジナルの解答に必要なライブラリーをインストール
# !pip install --upgrade pip
# !pip install -U pandas numpy scikit-learn imbalanced-learn

# 演習問題

> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [ ]:
df_recipt = df_receipt[['sales_ymd','customer_id','product_cd','amount']].rename(columns={'sales_ymd' : 'sales_date'})
df_recipt.head(10)

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id=="CS018205000001"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id == "CS018205000001" & amount>=1000')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','quantity','amount']].query('customer_id =="CS018205000001" & (1000<=amount or quantity >=5)')

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id =="CS018205000001" & 1000 <= amount<=2000')

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id =="CS018205000001" & product_cd !="P071401019"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
df_store.query("prefecture_cd !='13' and floor_area <=900")

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
23,S13008,成城店,13,東京都,東京都世田谷区成城三丁目,トウキョウトセタガヤクセイジョウサンチョウメ,03-0123-4012,139.5966,35.63614,883.0
24,S13020,十条仲原店,13,東京都,東京都北区十条仲原三丁目,トウキョウトキタクジュウジョウナカハラサンチョウメ,03-0123-4019,139.7186,35.76686,801.0
25,S13043,南品川店,13,東京都,東京都品川区南品川三丁目,トウキョウトシナガワクミナミシナガワサンチョウメ,03-0123-4027,139.7436,35.60981,845.0
29,S13041,八王子店,13,東京都,東京都八王子市大塚,トウキョウトハチオウジシオオツカ,042-123-4026,139.4235,35.63787,810.0
30,S13017,高円寺南店,13,東京都,東京都杉並区高円寺南四丁目,トウキョウトスギナミクコウエンジミナミヨンチョウメ,03-0123-4016,139.6513,35.70273,841.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0
50,S13037,南砂一丁目店,13,東京都,東京都江東区南砂一丁目,トウキョウトコウトウクミナミスナイッチョウメ,03-0123-4023,139.8215,35.67898,814.0


---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_store.query('store_cd.str.startswith("S14")',engine='python').head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.query('customer_id.str.endswith("1")',engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [ ]:
df_store.query('address.str.contains("横浜市")',engine='python')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.query('status_cd.str.contains("^[A-F]",regex=True)',engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.query('status_cd.str.contains("[1-9]$",regex=True)',engine='python')

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
...,...,...,...,...,...,...,...,...,...,...,...
21943,CS002515000242,加瀬 はるみ,1,女性,1966-06-23,52,184-0015,東京都小金井市貫井北町**********,S13002,20170102,A-20100124-7
21946,CS034412000010,末永 まひる,1,女性,1971-07-23,47,213-0035,神奈川県川崎市高津区向ケ丘**********,S14034,20150327,3-20090322-5
21952,CS002513000310,柳田 美嘉,1,女性,1963-09-17,55,187-0022,東京都小平市上水本町**********,S13002,20160911,1-20100711-2
21955,CS001413000156,上原 恵望子,1,女性,1974-12-10,44,144-0056,東京都大田区西六郷**********,S13001,20150724,1-20080210-1


---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.query('status_cd.str.contains("^[A-F].*[1-9]$",regex=True)',engine='python')

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
...,...,...,...,...,...,...,...,...,...,...,...
21767,CS001412000160,長沢 優,1,女性,1977-11-30,41,210-0814,神奈川県川崎市川崎区台町**********,S13001,20150907,E-20090412-7
21800,CS005315000066,松井 昌代,1,女性,1984-07-06,34,166-0001,東京都杉並区阿佐谷北**********,S13005,20150705,A-20080913-1
21852,CS016513000188,島崎 美幸,1,女性,1963-11-08,55,184-0011,東京都小金井市東町**********,S13016,20151008,B-20090809-3
21916,CS040615000030,梶原 千夏,1,女性,1952-08-31,66,226-0028,神奈川県横浜市緑区いぶき野**********,S14040,20150103,C-20100503-9


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
df_store.query('tel_no.str.contains("[0-9]{3}-[0-9]{3}-[0-9]{4}",regex=True)',engine='python')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.sort_values('birth_day', ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.sort_values('birth_day',ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
df_tmp = pd.concat([df_receipt[['customer_id','amount']],
                  df_receipt['amount'].rank(method = 'min',ascending=False)],axis=1)
df_tmp.columns = ['customer_id','amount', 'ranking']
df_tmp.sort_values('ranking',ascending=True).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
df_tmp = pd.concat([df_receipt[['customer_id','amount']],
                  df_receipt['amount'].rank(method = 'first',ascending=False)],axis=1)
df_tmp.columns = ['customer_id','amount', 'ranking']
df_tmp.sort_values('ranking',ascending=True).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,5.0
72747,ZZ000000000000,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
596,CS015515000083,5280,9.0
11275,CS017414000114,5280,10.0


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [ ]:
len(df_receipt)

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
df_receipt.customer_id.nunique()

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
df_receipt.groupby('store_cd').agg({'amount':'sum','quantity' : 'sum'}).reset_index()

,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupby('customer_id').agg({'sales_ymd':'max'}).reset_index()

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
...,...,...
8302,CS051513000004,20190719
8303,CS051515000002,20191025
8304,CS052212000002,20191017
8305,CS052514000001,20190822


In [ ]:
df_receipt.groupby('customer_id').sales_ymd.max().reset_index()

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
...,...,...
8302,CS051513000004,20190719
8303,CS051515000002,20191025
8304,CS052212000002,20191017
8305,CS052514000001,20190822


---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupby('customer_id').agg({'sales_ymd':'min'}).reset_index()

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207
...,...,...
8302,CS051513000004,20190719
8303,CS051515000002,20191025
8304,CS052212000002,20191017
8305,CS052514000001,20190822


In [ ]:
df_receipt.groupby('customer_id').sales_ymd.min().reset_index()

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207
...,...,...
8302,CS051513000004,20190719
8303,CS051515000002,20191025
8304,CS052212000002,20191017
8305,CS052514000001,20190822


---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
df_tmp = df_receipt.groupby('customer_id').agg({'sales_ymd' : ['min', 'max']}).reset_index()
df_tmp.columns = ['_'.join(pair) for pair in df_tmp]
df_tmp.query('sales_ymd_min != sales_ymd_max').head(10)



,customer_id_,sales_ymd_min,sales_ymd_max
1,CS001114000005,20180503,20190731
2,CS001115000010,20171228,20190405
3,CS001205000004,20170914,20190625
4,CS001205000006,20180207,20190224
13,CS001214000009,20170306,20190902
14,CS001214000017,20180828,20191006
16,CS001214000048,20171109,20190929
17,CS001214000052,20180208,20190617
20,CS001215000005,20170206,20181021
21,CS001215000040,20170214,20171022


---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').agg({'amount': 'mean'}).reset_index().sort_values('amount', ascending=False).head(5)

,store_cd,amount
28,S13052,402.867470
12,S13015,351.111960
7,S13003,350.915519
30,S14010,348.791262
5,S13001,348.470386


---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').agg({'amount' : 'median'}).reset_index().sort_values('amount',ascending=False).head(5)

,store_cd,amount
28,S13052,190.0
30,S14010,188.0
51,S14050,185.0
44,S14040,180.0
7,S13003,180.0


---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [ ]:
df_receipt.groupby('store_cd').product_cd.apply(lambda x : x.mode()).reset_index()

,store_cd,level_1,product_cd
0,S12007,0,P060303001
1,S12013,0,P060303001
2,S12014,0,P060303001
3,S12029,0,P060303001
4,S12030,0,P060303001
5,S13001,0,P060303001
6,S13002,0,P060303001
7,S13003,0,P071401001
8,S13004,0,P060303001
9,S13005,0,P040503001


---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').amount.var(ddof=0).reset_index().sort_values('amount',ascending=True)

,store_cd,amount
51,S14050,157698.130359
45,S14042,172061.187082
50,S14049,180361.837396
4,S12030,185542.898104
26,S13044,189597.952968
23,S13039,192184.721032
3,S12029,194078.594456
21,S13037,194914.825343
32,S14012,197658.564411
37,S14025,198167.977160


---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').amount.std(ddof=0).reset_index().sort_values('amount',ascending=False).head()

,store_cd,amount
28,S13052,663.391816
31,S14011,553.456916
42,S14034,544.903736
5,S13001,543.536561
12,S13015,543.409938


---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
df_tmp = df_receipt[['amount']].quantile(np.arange(5)/4)
df_tmp



,amount
0.00,10.0
0.25,102.0
0.50,170.0
0.75,288.0
1.00,10925.0


---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [ ]:
df_receipt.groupby('store_cd').amount.mean().reset_index().query('amount >= 330')

,store_cd,amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [ ]:
df_receipt.query('not customer_id.str.startswith("Z")',engine = 'python').groupby('customer_id').amount.sum().mean()

2547.742234529256

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [ ]:
amount_mean = df_receipt.query('not customer_id.str.startswith("Z")',engine ='python').groupby('customer_id').amount.sum().mean()
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_amount_sum[df_amount_sum['amount']>=amount_mean].head(10)



,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300


---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [ ]:
pd.merge(df_receipt, df_store[['store_cd','store_name']],how='inner', on='store_cd').head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
5,20181119,1542585600,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店
6,20171211,1512950400,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店
7,20191021,1571616000,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店
8,20170710,1499644800,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店
9,20190805,1564963200,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店


In [ ]:
df_receipt.head(3)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170


In [ ]:
df_store.head(3)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0


---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [ ]:
df_product.head(3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0


In [ ]:
df_category.head(3)

,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
0,4,惣菜,401,御飯類,40101,弁当類
1,4,惣菜,401,御飯類,40102,寿司類
2,4,惣菜,402,佃煮類,40201,魚介佃煮類


In [ ]:
pd.merge(df_product,
         df_category[['category_major_cd','category_medium_cd','category_small_cd','category_small_name']],
         how = 'inner', on = ['category_major_cd','category_medium_cd','category_small_cd']).head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,4,401,40101,198.0,149.0,弁当類
1,P040101002,4,401,40101,218.0,164.0,弁当類
2,P040101003,4,401,40101,230.0,173.0,弁当類
3,P040101004,4,401,40101,248.0,186.0,弁当類
4,P040101005,4,401,40101,268.0,201.0,弁当類
5,P040101006,4,401,40101,298.0,224.0,弁当類
6,P040101007,4,401,40101,338.0,254.0,弁当類
7,P040101008,4,401,40101,420.0,315.0,弁当類
8,P040101009,4,401,40101,498.0,374.0,弁当類
9,P040101010,4,401,40101,580.0,435.0,弁当類


---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [ ]:
df_customer.head(2)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0


In [ ]:
df_receipt.head(2)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81


In [ ]:
amount_ = df_receipt.query('not customer_id.str.startswith("Z")',engine ='python').groupby('customer_id').amount.sum().reset_index()
# amount_
gender_ = df_customer.query('gender_cd ==1')
# gender_
y = pd.merge(amount_,gender_['customer_id'],how = 'left',on='customer_id').fillna(0).head(10)
y



,customer_id,amount
0,CS001113000004,1298
1,CS001114000005,626
2,CS001115000010,3044
3,CS001205000004,1988
4,CS001205000006,3337
5,CS001211000025,456
6,CS001212000027,448
7,CS001212000031,296
8,CS001212000046,228
9,CS001212000070,456


In [ ]:
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_tmp = df_customer.query('not customer_id.str.startswith("Z") and gender_cd ==1 ',engine='python')
# df_amount_sum
y = pd.merge(df_tmp['customer_id'],df_amount_sum,how='left',on='customer_id').fillna(0).head(10)
y



,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [ ]:
df_receipt.head(2)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81


In [ ]:
df_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_sum = df_sum.query('not customer_id.str.startswith("Z")',engine='python')
df_sum = df_sum.sort_values('amount',ascending=False)

df_cnt = df_receipt[~df_receipt.duplicated(subset=['customer_id','sales_ymd'])]
df_cnt = df_cnt.query('not customer_id.str.startswith("Z")',engine='python')
df_cnt = df_cnt.groupby('customer_id').sales_ymd.count().reset_index()
df_cnt = df_cnt.sort_values('sales_ymd',ascending=False)
pd.merge(df_sum,df_cnt,how='outer',on='customer_id')

,customer_id,amount,sales_ymd
0,CS017415000097,23086,20
1,CS015415000185,20153,22
2,CS031414000051,19202,19
3,CS028415000007,19127,21
4,CS001605000009,18925,9
...,...,...,...
8301,CS035304000008,105,1
8302,CS005412000085,98,1
8303,CS010803000005,93,1
8304,CS002513000267,80,1


---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [ ]:
df_store_tmp = df_store.copy()
df_product_tmp = df_product.copy()

df_store_tmp['key'] = 0
df_product_tmp['key'] = 0

len(pd.merge(df_store_tmp, df_product_tmp,how='outer',on='key'))



531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [ ]:
df_receipt

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
...,...,...,...,...,...,...,...,...,...
104676,20180221,1519171200,S13043,1132,2,ZZ000000000000,P050101001,1,40
104677,20190911,1568160000,S14047,1132,2,ZZ000000000000,P071006005,1,218
104678,20170311,1489190400,S14040,1122,1,CS040513000195,P050405003,1,168
104679,20170331,1490918400,S13002,1142,1,CS002513000049,P060303001,1,148


In [ ]:
data = df_receipt[['amount','sales_ymd']].groupby('sales_ymd').sum().reset_index()

data = pd.concat([data, data.shift()],axis=1)
data.columns=['sales_ymd','amount','lag_ymd','lag_amount']
data
data['diff_amount'] = data['amount'] - data['lag_amount']
data.head(10)


,sales_ymd,amount,lag_ymd,lag_amount,diff_amount
0,20170101,33723,NaN,NaN,NaN
1,20170102,24165,20170101.0,33723.0,-9558.0
2,20170103,27503,20170102.0,24165.0,3338.0
3,20170104,36165,20170103.0,27503.0,8662.0
4,20170105,37830,20170104.0,36165.0,1665.0
5,20170106,32387,20170105.0,37830.0,-5443.0
6,20170107,23415,20170106.0,32387.0,-8972.0
7,20170108,24737,20170107.0,23415.0,1322.0
8,20170109,26718,20170108.0,24737.0,1981.0
9,20170110,20143,20170109.0,26718.0,-6575.0


---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [ ]:
# 縦
data = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
for i in range(1,4):
    if i == 1:
        df_lag = pd.concat([data,data.shift(i)],axis=1)
    else:
        df_lag = df_lag.append(pd.concat([data,data.shift(i)],axis=1))
df_lag

df_lag.columns=['sales_ymd','amount','lag_ymd','lag_amount']
df_lag
df_lag.dropna().sort_values(['sales_ymd','lag_ymd'],ascending=False).head()



,sales_ymd,amount,lag_ymd,lag_amount
1033,20191031,25216,20191030.0,26602.0
1033,20191031,25216,20191029.0,36091.0
1033,20191031,25216,20191028.0,40161.0
1032,20191030,26602,20191029.0,36091.0
1032,20191030,26602,20191028.0,40161.0


In [ ]:
#横
data = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
for i in range(1,4):
    if i==1:
        df_lag = pd.concat([data,data.shift(i)],axis=1)
    else:
        df_lag = pd.concat([df_lag, data.shift(i)],axis=1)

df_lag.columns = ['sales_ymd','amount','sales_ymd_1','amount_1','sales_ymd_2','amount_2','sales_ymd_3','amount_3']
df_lag.dropna()


,sales_ymd,amount,sales_ymd_1,amount_1,sales_ymd_2,amount_2,sales_ymd_3,amount_3
3,20170104,36165,20170103.0,27503.0,20170102.0,24165.0,20170101.0,33723.0
4,20170105,37830,20170104.0,36165.0,20170103.0,27503.0,20170102.0,24165.0
5,20170106,32387,20170105.0,37830.0,20170104.0,36165.0,20170103.0,27503.0
6,20170107,23415,20170106.0,32387.0,20170105.0,37830.0,20170104.0,36165.0
7,20170108,24737,20170107.0,23415.0,20170106.0,32387.0,20170105.0,37830.0
...,...,...,...,...,...,...,...,...
1029,20191027,37484,20191026.0,51771.0,20191025.0,28833.0,20191024.0,31868.0
1030,20191028,40161,20191027.0,37484.0,20191026.0,51771.0,20191025.0,28833.0
1031,20191029,36091,20191028.0,40161.0,20191027.0,37484.0,20191026.0,51771.0
1032,20191030,26602,20191029.0,36091.0,20191028.0,40161.0,20191027.0,37484.0


---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [ ]:
df_tmp = pd.merge(df_receipt, df_customer, on='customer_id',how='inner')
df_tmp['era'] =df_tmp['age'].apply(lambda x : math.floor(x/10)*10)
df_sales_summary = pd.pivot_table(df_tmp, index ='era', columns = 'gender_cd', values='amount',aggfunc='sum').reset_index()
df_sales_summary.columns=['era','male','female','unknown']
df_sales_summary



,era,male,female,unknown
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0
5,60,272469.0,987741.0,71418.0
6,70,13435.0,29764.0,2427.0
7,80,46360.0,262923.0,5111.0
8,90,NaN,6260.0,NaN


---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [ ]:
df_sales_summary = df_sales_summary.set_index('era').stack().reset_index().replace({
    'female' : '01',
    'male' : '00',
    'unknown' : '99'
}).rename(columns = {'level_1' : 'gender_cd', 0 : 'amount'})
df_sales_summary

,era,gender_cd,amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_customer['customer_id'],pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%m%d')],axis=1).head(10).head(10)

,customer_id,birth_day
0,CS021313000114,19810429
1,CS037613000071,19520401
2,CS031415000172,19761004
3,CS028811000001,19330327
4,CS001215000145,19950329
5,CS020401000016,19740915
6,CS015414000103,19770809
7,CS029403000008,19730817
8,CS015804000004,19310502
9,CS033513000180,19620711


---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_customer['customer_id'],  pd.to_datetime(df_customer['application_date'])],axis=1).head(10)

,customer_id,application_date
0,CS021313000114,1970-01-01 00:00:00.020150905
1,CS037613000071,1970-01-01 00:00:00.020150414
2,CS031415000172,1970-01-01 00:00:00.020150529
3,CS028811000001,1970-01-01 00:00:00.020160115
4,CS001215000145,1970-01-01 00:00:00.020170605
5,CS020401000016,1970-01-01 00:00:00.020150225
6,CS015414000103,1970-01-01 00:00:00.020150722
7,CS029403000008,1970-01-01 00:00:00.020150515
8,CS015804000004,1970-01-01 00:00:00.020150607
9,CS033513000180,1970-01-01 00:00:00.020150728


In [ ]:
type(pd.to_datetime(df_customer['application_date']))

pandas.core.series.Series

---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.to_datetime(df_receipt['sales_ymd']).astype('str')

0         1970-01-01 00:00:00.020181103
1         1970-01-01 00:00:00.020181118
2         1970-01-01 00:00:00.020170712
3         1970-01-01 00:00:00.020190205
4         1970-01-01 00:00:00.020180821
                      ...              
104676    1970-01-01 00:00:00.020180221
104677    1970-01-01 00:00:00.020190911
104678    1970-01-01 00:00:00.020170311
104679    1970-01-01 00:00:00.020170331
104680    1970-01-01 00:00:00.020190423
Name: sales_ymd, Length: 104681, dtype: object

In [ ]:
type(df_receipt['receipt_no'])

pandas.core.series.Series

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],
                   pd.to_datetime(df_receipt['sales_ymd'].astype('str'))],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s')],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.year],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018
1,1132,2,2018
2,1102,1,2017
3,1132,1,2019
4,1102,2,2018
5,1112,1,2019
6,1102,2,2018
7,1102,1,2019
8,1112,2,2017
9,1102,1,2019


---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.strftime('%m')],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
5,1112,1,06
6,1102,2,12
7,1102,1,09
8,1112,2,05
9,1102,1,10


In [ ]:
# dt.monthでは0埋め2桁で取り出せない
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.month],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,7
3,1132,1,2
4,1102,2,8
5,1112,1,6
6,1102,2,12
7,1102,1,9
8,1112,2,5
9,1102,1,10


---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.strftime('%d')],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
5,1112,1,05
6,1102,2,05
7,1102,1,22
8,1112,2,04
9,1102,1,10


---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')
df_tmp = df_tmp.groupby('customer_id').amount.sum().reset_index()
df_tmp['sales_flg'] = df_tmp['amount'].apply(lambda x : 1 if x > 2000 else 0)
df_tmp

,customer_id,amount,sales_flg
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
...,...,...,...
8301,CS051212000001,336,0
8302,CS051513000004,551,0
8303,CS051515000002,265,0
8304,CS052212000002,192,0



> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [ ]:
df_tmp = df_customer[['customer_id','postal_cd']].copy()
df_tmp['postal_flg'] = df_tmp['postal_cd'].apply(lambda x : 1 if 100 <=int(x[0:3]) <=209 else 0)
pd.merge(df_tmp, df_receipt,how='inner',on='customer_id').groupby('postal_flg').agg({'customer_id' : 'nunique'})

,customer_id
postal_flg,
0,3906
1,4400


---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [ ]:
pd.concat([df_customer[['customer_id','address']],df_customer['address'].str[0:3].map({
    '埼玉県':11,
    '千葉県':12,
    '東京都':13,
    '神奈川':14
})],axis=1).head(10)

,customer_id,address,address
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [ ]:
df_tmp = df_receipt.groupby('customer_id').amount.sum().reset_index()

pct25 = np.quantile(df_tmp['amount'], 0.25)
pct50 = np.quantile(df_tmp['amount'], 0.50)
pct75 = np.quantile(df_tmp['amount'], 0.75)

def pct_group(x):
    if x < pct25:    
        return 1
    elif pct25 <= x < pct50:
        return 2
    elif pct50 <= x < pct75:
        return 3
    else:
        return 4

df_tmp['pct_group'] = df_tmp['amount'].apply(lambda x : pct_group(x))
df_tmp.head(10)




,customer_id,amount,pct_group
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3
5,CS001211000025,456,1
6,CS001212000027,448,1
7,CS001212000031,296,1
8,CS001212000046,228,1
9,CS001212000070,456,1


---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [ ]:
df_customer_era = pd.concat([df_customer[['customer_id', 'birth_day']],df_customer['age'].apply(lambda x : min(math.floor(x / 10)*10, 60))],axis=1 )
df_customer_era

,customer_id,birth_day,age
0,CS021313000114,1981-04-29,30
1,CS037613000071,1952-04-01,60
2,CS031415000172,1976-10-04,40
3,CS028811000001,1933-03-27,60
4,CS001215000145,1995-03-29,20
...,...,...,...
21966,CS002512000474,1959-10-12,50
21967,CS029414000065,1970-10-19,40
21968,CS012403000043,1972-12-16,40
21969,CS033512000184,1964-06-05,50



> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [ ]:
df_customer_gender=df_customer[['customer_id','gender_cd']]
pd.merge(df_customer_era , df_customer_gender, how='inner', on='customer_id')

,customer_id,birth_day,age,gender_cd
0,CS021313000114,1981-04-29,30,1
1,CS037613000071,1952-04-01,60,9
2,CS031415000172,1976-10-04,40,1
3,CS028811000001,1933-03-27,60,1
4,CS001215000145,1995-03-29,20,1
...,...,...,...,...
21966,CS002512000474,1959-10-12,50,1
21967,CS029414000065,1970-10-19,40,1
21968,CS012403000043,1972-12-16,40,0
21969,CS033512000184,1964-06-05,50,1


In [ ]:
df_customer_era['era_gender'] =df_customer['gender_cd'].astype('str') + df_customer_era['age'].astype('str')
df_customer_era

,customer_id,birth_day,age,era_gender
0,CS021313000114,1981-04-29,30,130
1,CS037613000071,1952-04-01,60,960
2,CS031415000172,1976-10-04,40,140
3,CS028811000001,1933-03-27,60,160
4,CS001215000145,1995-03-29,20,120
...,...,...,...,...
21966,CS002512000474,1959-10-12,50,150
21967,CS029414000065,1970-10-19,40,140
21968,CS012403000043,1972-12-16,40,040
21969,CS033512000184,1964-06-05,50,150


---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [ ]:
pd.get_dummies(df_customer[['customer_id','gender_cd']],columns=['gender_cd']).head(10)

,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0
5,CS020401000016,1,0,0
6,CS015414000103,0,1,0
7,CS029403000008,1,0,0
8,CS015804000004,1,0,0
9,CS033513000180,0,1,0


---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').groupby('customer_id').amount.sum().reset_index()
scaler = preprocessing.StandardScaler()
df_tmp['amount_scaler'] = scaler.fit_transform(df_tmp[['amount']])
df_tmp

,customer_id,amount,amount_scaler
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
...,...,...,...
8301,CS051212000001,336,-0.812988
8302,CS051513000004,551,-0.733959
8303,CS051515000002,265,-0.839086
8304,CS052212000002,192,-0.865919


In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_scaler'] = preprocessing.scale(df_tmp['amount'])
df_tmp


,customer_id,amount,amount_scaler
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
...,...,...,...
8301,CS051212000001,336,-0.812988
8302,CS051513000004,551,-0.733959
8303,CS051515000002,265,-0.839086
8304,CS052212000002,192,-0.865919


---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_minmax'] = preprocessing.minmax_scale(df_tmp['amount'])
df_tmp


,customer_id,amount,amount_minmax
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
...,...,...,...
8301,CS051212000001,336,0.011557
8302,CS051513000004,551,0.020899
8303,CS051515000002,265,0.008472
8304,CS052212000002,192,0.005301


In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').groupby('customer_id').amount.sum().reset_index()
minmax_scaler = preprocessing.MinMaxScaler()
df_tmp['min_max'] = minmax_scaler.fit_transform(df_tmp[['amount']])
df_tmp
                                                

,customer_id,amount,min_max
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
...,...,...,...
8301,CS051212000001,336,0.011557
8302,CS051513000004,551,0.020899
8303,CS051515000002,265,0.008472
8304,CS052212000002,192,0.005301


---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').\
groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_log'] = np.log10(df_tmp['amount']+1)
df_tmp

,customer_id,amount,amount_log
0,CS001113000004,1298,3.113609
1,CS001114000005,626,2.797268
2,CS001115000010,3044,3.483587
3,CS001205000004,1988,3.298635
4,CS001205000006,3337,3.523486
...,...,...,...
8301,CS051212000001,336,2.527630
8302,CS051513000004,551,2.741939
8303,CS051515000002,265,2.424882
8304,CS052212000002,192,2.285557


---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_log'] = np.log(df_tmp['amount'])
df_tmp
                

,customer_id,amount,amount_log
0,CS001113000004,1298,7.168580
1,CS001114000005,626,6.439350
2,CS001115000010,3044,8.020928
3,CS001205000004,1988,7.594884
4,CS001205000006,3337,8.112827
...,...,...,...
8301,CS051212000001,336,5.817111
8302,CS051513000004,551,6.311735
8303,CS051515000002,265,5.579730
8304,CS052212000002,192,5.257495


---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_product.copy()
df_tmp['unit_profit'] = df_tmp['unit_price'] - df_tmp['unit_cost']
df_tmp

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_profit
0,P040101001,4,401,40101,198.0,149.0,49.0
1,P040101002,4,401,40101,218.0,164.0,54.0
2,P040101003,4,401,40101,230.0,173.0,57.0
3,P040101004,4,401,40101,248.0,186.0,62.0
4,P040101005,4,401,40101,268.0,201.0,67.0
...,...,...,...,...,...,...,...
10025,P091503001,9,915,91503,280.0,210.0,70.0
10026,P091503002,9,915,91503,680.0,510.0,170.0
10027,P091503003,9,915,91503,1080.0,810.0,270.0
10028,P091503004,9,915,91503,1130.0,848.0,282.0



> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [ ]:
# 単価:300
# 原価:100
# 粗利率
((300-100)/300)*100 # 66%

66.66666666666666

In [ ]:
df_tmp = df_product.copy()
df_tmp['rate'] = (df_tmp['unit_price']-df_tmp['unit_cost']) / df_tmp['unit_price']
df_tmp['rate'].mean(skipna=True)

0.24911389885176904

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
# df_tmp
df_tmp['new_price'] = df_tmp['unit_cost'].apply(lambda x : np.floor(x/0.7))
df_tmp['new_price_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_price_rate
0,P040101001,4,401,40101,198.0,149.0,212.0,0.297170
1,P040101002,4,401,40101,218.0,164.0,234.0,0.299145
2,P040101003,4,401,40101,230.0,173.0,247.0,0.299595
3,P040101004,4,401,40101,248.0,186.0,265.0,0.298113
4,P040101005,4,401,40101,268.0,201.0,287.0,0.299652
...,...,...,...,...,...,...,...,...
10025,P091503001,9,915,91503,280.0,210.0,300.0,0.300000
10026,P091503002,9,915,91503,680.0,510.0,728.0,0.299451
10027,P091503003,9,915,91503,1080.0,810.0,1157.0,0.299914
10028,P091503004,9,915,91503,1130.0,848.0,1211.0,0.299752


---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_cost'].apply(lambda x : np.ceil(x/0.7))
df_tmp['new_price_rate'] = (df_tmp['new_price']-df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_price_rate
0,P040101001,4,401,40101,198.0,149.0,213.0,0.300469
1,P040101002,4,401,40101,218.0,164.0,235.0,0.302128
2,P040101003,4,401,40101,230.0,173.0,248.0,0.302419
3,P040101004,4,401,40101,248.0,186.0,266.0,0.300752
4,P040101005,4,401,40101,268.0,201.0,288.0,0.302083
...,...,...,...,...,...,...,...,...
10025,P091503001,9,915,91503,280.0,210.0,300.0,0.300000
10026,P091503002,9,915,91503,680.0,510.0,729.0,0.300412
10027,P091503003,9,915,91503,1080.0,810.0,1158.0,0.300518
10028,P091503004,9,915,91503,1130.0,848.0,1212.0,0.300330


---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
df_tmp['tax'] = df_tmp['unit_price']*1.1
df_tmp


,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,tax
0,P040101001,4,401,40101,198.0,149.0,217.8
1,P040101002,4,401,40101,218.0,164.0,239.8
2,P040101003,4,401,40101,230.0,173.0,253.0
3,P040101004,4,401,40101,248.0,186.0,272.8
4,P040101005,4,401,40101,268.0,201.0,294.8
...,...,...,...,...,...,...,...
10025,P091503001,9,915,91503,280.0,210.0,308.0
10026,P091503002,9,915,91503,680.0,510.0,748.0
10027,P091503003,9,915,91503,1080.0,810.0,1188.0
10028,P091503004,9,915,91503,1130.0,848.0,1243.0


---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [ ]:
df_tmp = df_receipt.merge(df_product, how='left',on='product_cd').groupby(['customer_id','category_major_cd']).amount.sum()

df_tmp

customer_id     category_major_cd
CS001113000004  7                       1298
CS001114000005  5                         40
                7                        486
                8                        100
CS001115000010  7                       2694
                                      ...   
ZZ000000000000  5                    1281969
                6                    1150203
                7                    6943009
                8                     544125
                9                    1341264
Name: amount, Length: 28230, dtype: int64

In [ ]:
df_tmp = df_receipt.merge(df_product, how='left',on='product_cd').groupby(['customer_id','category_major_cd']).amount.sum().unstack()
df_tmp
df_tmp =df_tmp[df_tmp[7]>0]
df_tmp['sum'] = df_tmp.sum(axis=1)
df_tmp['07_rate'] = df_tmp[7] /df_tmp['sum']
df_tmp

category_major_cd,4,5,6,7,8,9,sum,07_rate
customer_id,,,,,,,,
CS001113000004,NaN,NaN,NaN,1298.0,NaN,NaN,1298.0,1.000000
CS001114000005,NaN,40.0,NaN,486.0,100.0,NaN,626.0,0.776358
CS001115000010,NaN,NaN,NaN,2694.0,NaN,350.0,3044.0,0.885020
CS001205000004,100.0,128.0,286.0,346.0,368.0,760.0,1988.0,0.174044
CS001205000006,635.0,60.0,198.0,2004.0,80.0,360.0,3337.0,0.600539
...,...,...,...,...,...,...,...,...
CS051212000001,268.0,NaN,NaN,68.0,NaN,NaN,336.0,0.202381
CS051513000004,318.0,NaN,NaN,233.0,NaN,NaN,551.0,0.422868
CS052212000002,NaN,NaN,NaN,102.0,90.0,NaN,192.0,0.531250


---
>P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
df_customer.head(2)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0


In [ ]:
df_tmp = pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],how='inner',on='customer_id')
df_tmp = df_tmp.drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
df_tmp['elapsed_date'] = df_tmp['sales_ymd'] - df_tmp['application_date']
df_tmp


,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,1970-01-01 00:00:00.020150201,17837 days 23:59:59.979849799
1,CS006214000001,2017-05-09,1970-01-01 00:00:00.020150201,17294 days 23:59:59.979849799
2,CS006214000001,2017-06-08,1970-01-01 00:00:00.020150201,17324 days 23:59:59.979849799
4,CS006214000001,2018-10-28,1970-01-01 00:00:00.020150201,17831 days 23:59:59.979849799
7,CS006214000001,2019-09-08,1970-01-01 00:00:00.020150201,18146 days 23:59:59.979849799
...,...,...,...,...
65672,CS004411000027,2017-06-01,1970-01-01 00:00:00.020150517,17317 days 23:59:59.979849483
65674,CS040513000029,2018-06-07,1970-01-01 00:00:00.020150915,17688 days 23:59:59.979849085
65676,CS004613000146,2017-12-02,1970-01-01 00:00:00.020160813,17501 days 23:59:59.979839187
65678,CS002314000037,2018-04-21,1970-01-01 00:00:00.020151202,17641 days 23:59:59.979848798


---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
df_tmp = pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],how='inner',on='customer_id')
df_tmp = df_tmp.drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
df_tmp['elapsed_date'] = df_tmp[['sales_ymd' , 'application_date']].apply(lambda x : 
                    relativedelta(x[0],x[1]).years*12 + relativedelta(x[0],x[1]).months,axis=1)
df_tmp
                                                                           


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,1970-01-01 00:00:00.020150201,586
1,CS006214000001,2017-05-09,1970-01-01 00:00:00.020150201,568
2,CS006214000001,2017-06-08,1970-01-01 00:00:00.020150201,569
4,CS006214000001,2018-10-28,1970-01-01 00:00:00.020150201,585
7,CS006214000001,2019-09-08,1970-01-01 00:00:00.020150201,596
...,...,...,...,...
65672,CS004411000027,2017-06-01,1970-01-01 00:00:00.020150517,568
65674,CS040513000029,2018-06-07,1970-01-01 00:00:00.020150915,581
65676,CS004613000146,2017-12-02,1970-01-01 00:00:00.020160813,575
65678,CS002314000037,2018-04-21,1970-01-01 00:00:00.020151202,579


---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
df_customer.head(2)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0


In [ ]:
df_tmp = pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],how='inner',on='customer_id')
df_tmp = df_tmp.drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
df_tmp['elapsed_date'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x : relativedelta(x[0],x[1]).years,axis=1)
df_tmp                                                                       


,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,1970-01-01 00:00:00.020150201,48
1,CS006214000001,2017-05-09,1970-01-01 00:00:00.020150201,47
2,CS006214000001,2017-06-08,1970-01-01 00:00:00.020150201,47
4,CS006214000001,2018-10-28,1970-01-01 00:00:00.020150201,48
7,CS006214000001,2019-09-08,1970-01-01 00:00:00.020150201,49
...,...,...,...,...
65672,CS004411000027,2017-06-01,1970-01-01 00:00:00.020150517,47
65674,CS040513000029,2018-06-07,1970-01-01 00:00:00.020150915,48
65676,CS004613000146,2017-12-02,1970-01-01 00:00:00.020160813,47
65678,CS002314000037,2018-04-21,1970-01-01 00:00:00.020151202,48


---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
df_tmp = pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],how='inner',on='customer_id')
df_tmp = df_tmp.drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
df_tmp['elapsed_date'] =(df_tmp['sales_ymd'].astype(np.int64) / 10**9 ) -\
(df_tmp['application_date'].astype(np.int64) / 10**9)
df_tmp


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  """


,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,1970-01-01 00:00:00.020150201,1.541203e+09
1,CS006214000001,2017-05-09,1970-01-01 00:00:00.020150201,1.494288e+09
2,CS006214000001,2017-06-08,1970-01-01 00:00:00.020150201,1.496880e+09
4,CS006214000001,2018-10-28,1970-01-01 00:00:00.020150201,1.540685e+09
7,CS006214000001,2019-09-08,1970-01-01 00:00:00.020150201,1.567901e+09
...,...,...,...,...
65672,CS004411000027,2017-06-01,1970-01-01 00:00:00.020150517,1.496275e+09
65674,CS040513000029,2018-06-07,1970-01-01 00:00:00.020150915,1.528330e+09
65676,CS004613000146,2017-12-02,1970-01-01 00:00:00.020160813,1.512173e+09
65678,CS002314000037,2018-04-21,1970-01-01 00:00:00.020151202,1.524269e+09


---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
df_receipt.head(2)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81


In [ ]:
df_tmp = df_receipt[['customer_id', 'sales_ymd']]
df_tmp = df_tmp.drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['monday'] = df_tmp['sales_ymd'].apply(lambda x : x - relativedelta(days=x.weekday()))
df_tmp['elapsed_date'] = df_tmp['sales_ymd'] - df_tmp['monday']
df_tmp


,customer_id,sales_ymd,monday,elapsed_date
0,CS006214000001,2018-11-03,2018-10-29,5 days
1,CS008415000097,2018-11-18,2018-11-12,6 days
2,CS028414000014,2017-07-12,2017-07-10,2 days
3,ZZ000000000000,2019-02-05,2019-02-04,1 days
4,CS025415000050,2018-08-21,2018-08-20,1 days
...,...,...,...,...
103599,CS023514000014,2018-07-01,2018-06-25,6 days
103694,CS028514000011,2017-07-14,2017-07-10,4 days
103699,CS016515000072,2019-01-09,2019-01-07,2 days
103818,CS030415000050,2018-05-21,2018-05-21,0 days


---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [ ]:
df_customer.sample(frac=0.01).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4097,CS008414000008,関口 那奈,1,女性,1971-08-30,47,157-0072,東京都世田谷区祖師谷**********,S13008,20150514,C-20100129-C
8623,CS038213000030,水谷 しほり,1,女性,1997-01-12,22,279-0043,千葉県浦安市富士見**********,S13038,20141124,0-00000000-0
17659,CS018712000008,江藤 由樹,1,女性,1940-08-20,78,203-0012,東京都東久留米市浅間町**********,S13018,20150622,0-00000000-0
9967,CS034513000135,奥貫 由宇,1,女性,1966-04-08,52,216-0001,神奈川県川崎市宮前区野川**********,S14034,20160131,8-20090309-4
9664,CS035413000211,塚本 美菜,1,女性,1973-12-11,45,154-0015,東京都世田谷区桜新町**********,S13035,20150703,6-20100214-4
8339,CS035615000089,玉城 玲那,1,女性,1955-12-14,63,154-0015,東京都世田谷区桜新町**********,S13035,20150228,2-20080729-4
3553,CS035315000144,大村 なつみ,1,女性,1982-07-29,36,157-0073,東京都世田谷区砧**********,S13035,20150812,0-00000000-0
12999,CS005612000097,小田 幸子,1,女性,1949-05-07,69,168-0081,東京都杉並区宮前**********,S13005,20151106,0-00000000-0
11740,CS038315000152,本間 里穂,1,女性,1981-01-22,38,134-0084,東京都江戸川区東葛西**********,S13038,20150608,0-00000000-0
1214,CS004313000010,菊地 朝陽,1,女性,1985-08-04,33,165-0035,東京都中野区白鷺**********,S13004,20150124,0-00000000-0


---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [ ]:
_, df_tmp = train_test_split(df_customer,test_size=0.1, stratify=df_customer['gender'])
df_tmp = df_tmp.groupby('gender_cd').agg({'customer_id' : 'count'})
df_tmp


,customer_id
gender_cd,
0,298
1,1793
9,107


---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [ ]:
df_receipt

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_ss'] = preprocessing.scale(df_tmp['amount'])
df_tmp.query('abs(amount_ss) >= 3').head(10)

,customer_id,amount,amount_ss
332,CS001605000009,18925,6.019921
1755,CS006415000147,12723,3.740202
1817,CS006515000023,18372,5.816651
1833,CS006515000125,12575,3.685800
1841,CS006515000209,11373,3.243972
1870,CS007115000006,11528,3.300946
1941,CS007514000056,13293,3.949721
1943,CS007514000094,15735,4.847347
1951,CS007515000107,11188,3.175970
1997,CS007615000026,11959,3.459372


---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').groupby('customer_id').amount.sum().reset_index()

df_tmp 
pct75 = np.percentile(df_tmp['amount'],q=75)
pct25 = np.percentile(df_tmp['amount'],q=25)
iqr = pct75 - pct25
iqr
amount_low =pct25 -(1.5 *iqr)
amount_low
amount_hight =pct75 +(1.5 *iqr)
df_tmp.query('amount < @amount_low or @amount_hight < amount').head(10)


,customer_id,amount
98,CS001414000048,8584
332,CS001605000009,18925
549,CS002415000594,9568
1180,CS004414000181,9584
1558,CS005415000137,8734
1733,CS006414000001,9156
1736,CS006414000029,9179
1752,CS006415000105,10042
1755,CS006415000147,12723
1757,CS006415000157,10648


---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [ ]:
df_tmp = df_product.copy()
print('削除前',len(df_tmp))
df_tmp.dropna(inplace=True)
print('削除後',len(df_tmp))

削除前 10030
削除後 10023


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [11]:
df_product_2 = df_product.fillna({'unit_price':np.round(np.nanmean(df_product['unit_price'])),
                                  'unit_cost':np.round(np.nanmean(df_product['unit_cost']))})
df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
df_product_2          0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [14]:
df_product3 = df_product.fillna({'unit_price' : np.round(np.nanmedian(df_product['unit_price'])),
                                 'unit_cost' : np.round(np.nanmedian(df_product['unit_cost']))})
df_product3.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
df_product_2          0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [47]:
df_tmp = df_product.groupby('category_small_cd').agg({'unit_price':'median', 'unit_cost':'median'}).reset_index()
df_tmp.columns = ['category_small_cd', 'median_price', 'median_cost']

df_product_4 = pd.merge(df_product, df_tmp, how='inner', on='category_small_cd')
df_product_4['unit_price'] = df_product_4[['unit_price', 'median_price']].apply(lambda x : x[1] if np.isnan(x[0]) else x[1],axis=1)
df_product_4['unit_cost'] = df_product_4[['unit_cost', 'median_cost']].apply(lambda x : x[1] if np.isnan(x[0]) else x[1],axis=1)

df_product_4.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
df_product_2          0
median_price          0
median_cost           0
dtype: int64

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [ ]:
df_receipt
df_tmp_1 = df_receipt.query('20190101 <= sales_ymd <=20191231')
df_tmp_1 = pd.merge(df_customer['customer_id'],df_tmp_1[['customer_id','amount']],how='inner', on='customer_id').groupby('customer_id').sum().reset_index().rename(columns={'amount' : 'amount_2019'})
df_tmp_2 = pd.merge(df_customer['customer_id'], df_receipt[['customer_id', 'amount']],how='inner',on='customer_id').groupby('customer_id').amount.sum().reset_index()

df_tmp = pd.merge(df_tmp_1, df_tmp_2, on='customer_id', how='inner')
df_tmp
df_tmp['rate']= df_tmp['amount_2019'] / df_tmp['amount']
df_tmp.isnull().sum()

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [ ]:
df_geocode

In [71]:
df_customer_1 = pd.merge(df_customer[['customer_id', 'postal_cd']],df_geocode[['postal_cd','longitude','latitude']] ,how='inner',on='postal_cd')
df_customer_1 = df_customer_1.groupby('customer_id').agg({'longitude' : 'mean' , 'latitude':'mean'}).reset_index().rename(columns={'longitude' : 'm_longitude','latitude' : 'm_latitude'})
df_customer_1 = pd.merge(df_customer, df_customer_1,how='inner', on='customer_id')
df_customer_1.head(2)


,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,m_longitude,m_latitude
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.31779,35.41358
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.83502,35.67193


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [77]:
def calc_distance(x1, y1, x2, y2):
    distance = 6371 * math.acos(math.sin(math.radians(y1)) * math.sin(math.radians(y2))
    + math.cos(math.radians(y1))* math.cos(math.radians(y2)) * math.cos(math.radians(x1) - math.radians(x2)))

    return distance

df_tmp = pd.merge(df_customer_1,df_store, how='inner', left_on = 'application_store_cd' , right_on='store_cd')
# df_tmp.head(3)
df_tmp['distance'] = df_tmp[['m_longitude','m_latitude','longitude','latitude']].apply(
    lambda x : calc_distance(x[0],x[1],x[2],x[3]),axis=1)

df_tmp[['customer_id', 'address_x','address_y','distance']].head(3)

,customer_id,address_x,address_y,distance
0,CS021313000114,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
1,CS021313000025,神奈川県伊勢原市伊勢原**********,神奈川県伊勢原市伊勢原四丁目,0.474282
2,CS021411000096,神奈川県伊勢原市高森**********,神奈川県伊勢原市伊勢原四丁目,2.480155


---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [109]:
df_tmp = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_customer_u = pd.merge(df_customer, df_tmp,how='left',on='customer_id').sort_values(['amount','customer_id'],ascending=[False, True])
df_customer_u.drop_duplicates(subset=['customer_name', 'postal_cd'],keep='first',inplace=True)
len(df_customer) - len(df_customer_u)

30

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [122]:
df_customer_n =pd.merge(df_customer,df_customer_u[['customer_id','customer_name','postal_cd']], how='inner',on=['customer_name','postal_cd'])
df_customer_n.rename(columns={'customer_id_x' : 'customer_id','customer_id_y' : 'integration_id'}, inplace=True)
print('IDの差' , len(df_customer_n['customer_id'].unique()) -len(df_customer_n['integration_id'].unique()) )

IDの差 30


---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

In [159]:
# del df_customer_1
# del df_customer_n

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [163]:
df_tmp = pd.merge(df_customer, df_receipt,how='inner', on='customer_id')
df_train, df_test = train_test_split(df_tmp, test_size=0.2, shuffle=True,random_state=71)
print(f'【学習データ数】:{len(df_train)}')
print(f'【評価データ数】:{len(df_test)}')
print('-'*30)
print(f'【学習データ割合】:{len(df_train)/len(df_tmp)}')
print(f'【評価データ割合】:{len(df_test)/len(df_tmp)}')

【学習データ数】:52545
【評価データ数】:13137
------------------------------
【学習データ割合】:0.7999908650771901
【評価データ割合】:0.2000091349228099


---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [167]:
df_tmp = df_receipt[['sales_ymd','amount']].copy()
df_tmp['sales_ym'] = df_tmp['sales_ymd'].astype('str').str[0:6]
df_tmp = df_tmp.groupby('sales_ym').amount.sum().reset_index()

def split_data(df, train_size, test_size, slide_window, start_point):
    train_start = start_point * slide_window
    test_start = train_start + train_size
    return df[train_start : test_start], df[test_start : test_start + test_size]

df_train_1, df_test_1 = split_data(df_tmp, train_size = 12, test_size = 6, slide_window=6,start_point=0)
df_train_2, df_test_2 = split_data(df_tmp, train_size = 12, test_size = 6, slide_window=6,start_point=1)
df_train_3, df_test_3 = split_data(df_tmp, train_size = 12, test_size = 6, slide_window=6,start_point=2)
df_train_1

,sales_ym,amount
0,201701,902056
1,201702,764413
2,201703,962945
3,201704,847566
4,201705,884010
5,201706,894242
6,201707,959205
7,201708,954836
8,201709,902037
9,201710,905739


In [169]:
df_test_1

,sales_ym,amount
12,201801,944509
13,201802,864128
14,201803,946588
15,201804,937099
16,201805,1004438
17,201806,1012329


---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [183]:
df_tmp = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_tmp = pd.merge(df_customer, df_tmp, how='left', on='customer_id')
df_tmp['buy_flg'] = df_tmp['amount'].apply(lambda x: 0 if np.isnan(x) else 1)

print('0の件数', len(df_tmp.query('buy_flg == 0')))
print('1の件数', len(df_tmp.query('buy_flg == 1')))

positive_count = len(df_tmp.query('buy_flg == 1'))

rs = RandomUnderSampler(random_state=71)

df_sample, _ = rs.fit_sample(df_tmp, df_tmp.buy_flg)

print('0の件数', len(df_sample.query('buy_flg == 0')))
print('1の件数', len(df_sample.query('buy_flg == 1')))

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [187]:
df_customer[['gender_cd', 'gender']].drop_duplicates()
df_customer_s = df_customer.drop(columns='gender')
df_customer_s.head(3)

,customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C


---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [192]:
df_product_full =  pd.merge(df_product,df_category[['category_small_cd','category_major_name','category_medium_name','category_small_name']],
         how='inner', on='category_small_cd')

In [193]:
df_product_full.head(2)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,df_product_2,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,198.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,218.0,惣菜,御飯類,弁当類


---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [195]:
os.makedirs('./data')
df_product_full.to_csv('./data/P_df_product_full_UTF-8_header.csv',encoding='UTF-8', index=False)

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [196]:
df_product_full.to_csv('./data/P_df_product_full_CP932_header.csv',encoding='CP932',index=False)

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [197]:
df_product_full.to_csv('./data/P_df_product_full_UTF-8.csv',header=False , encoding='UTF-8',index=False)

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [198]:
df_tmp = pd.read_csv('/content/data/P_df_product_full_UTF-8_header.csv')
df_tmp.head(10)


,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,df_product_2,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,198.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,218.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,230.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,248.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,268.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,298.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,338.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,420.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,498.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,580.0,惣菜,御飯類,弁当類


---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [201]:
df_tmp = pd.read_csv('/content/data/P_df_product_full_UTF-8.csv',header=None)
df_tmp


,0,1,2,3,4,5,6,7,8,9
0,P040101001,4,401,40101,198.0,149.0,198.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,218.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,230.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,248.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,268.0,惣菜,御飯類,弁当類
...,...,...,...,...,...,...,...,...,...,...
10025,P091503001,9,915,91503,280.0,210.0,280.0,洗剤,ＤＩＹ用品,園芸用品
10026,P091503002,9,915,91503,680.0,510.0,680.0,洗剤,ＤＩＹ用品,園芸用品
10027,P091503003,9,915,91503,1080.0,810.0,1080.0,洗剤,ＤＩＹ用品,園芸用品
10028,P091503004,9,915,91503,1130.0,848.0,1130.0,洗剤,ＤＩＹ用品,園芸用品


---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [203]:
df_product_full.to_csv('./data/P_df_product_full_UTF-8_header.tsv',sep='\t',encoding='UTF-8',index=False)

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [207]:
df_tmp = pd.read_table('/content/data/P_df_product_full_UTF-8_header.tsv',encoding='UTF-8')
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,df_product_2,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,198.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,218.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,230.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,248.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,268.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,298.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,338.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,420.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,498.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,580.0,惣菜,御飯類,弁当類


## 終了